In [3]:
import time
 
from random import randint

from default import PROBES_FILE, ANCHORS_FILE, RIPE_CREDENTIALS
from utils.measurement_utils import load_pickle
from utils.helpers import distance
from measurements.src.ping import PING

NB_ANCHOR = 5
NB_VP = 10
NB_PACKETS = 5

## get measurement datasets

In [4]:
probes = load_pickle(PROBES_FILE)

anchors = load_pickle(ANCHORS_FILE)

In [5]:
target_anchors_index = [randint(0, len(anchors)) for _ in range(0, NB_ANCHOR)]

# get target dataset
target_anchors = {}
anchors_ip = list(anchors.keys())
for index in target_anchors_index:
    target_anchors[anchors_ip[index]] = anchors[anchors_ip[index]]

# get vantage points dataset
vp_anchors = {}
for probe, probe_description in anchors.items():
    if not probe in target_anchors:
        vp_anchors[probe] = probe_description

In [7]:
print("targets")
for index, (probe, probe_description) in enumerate(target_anchors.items()):
    print(f"target_ip: {probe} : {probe_description}")
print()

print("vps:")
for index, (probe, probe_description) in enumerate(vp_anchors.items()):
    if index > 10:
        break
    print(f"target_ip: {probe} : {probe_description}")

targets
target_ip: 207.246.88.250 : {'id': 6436, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.5605, 'longitude': -74.4815}
target_ip: 76.26.115.194 : {'id': 6072, 'is_anchor': True, 'country_code': 'US', 'latitude': 39.9585, 'longitude': -75.6095}
target_ip: 217.28.166.213 : {'id': 6952, 'is_anchor': True, 'country_code': 'DK', 'latitude': 56.1085, 'longitude': 10.1595}
target_ip: 81.211.201.45 : {'id': 6757, 'is_anchor': True, 'country_code': 'KZ', 'latitude': 52.2805, 'longitude': 76.9695}
target_ip: 185.151.70.11 : {'id': 6395, 'is_anchor': True, 'country_code': 'FR', 'latitude': 43.5395, 'longitude': 1.5215}

vps:
target_ip: 213.225.160.239 : {'id': 6025, 'is_anchor': True, 'country_code': 'FR', 'latitude': 48.5795, 'longitude': 7.7485}
target_ip: 145.220.0.55 : {'id': 6031, 'is_anchor': True, 'country_code': 'NL', 'latitude': 52.3675, 'longitude': 4.8985}
target_ip: 5.28.0.17 : {'id': 6039, 'is_anchor': True, 'country_code': 'HU', 'latitude': 47.5015, 'longitude': 19.03

## from ip target list select probes

In [8]:
def get_closest_vp(target_ip: str, target_description: dict, vp_anchors: dict, nb_vp: int):

    distance_probes_target = []
    for vp_ip, vp_description in vp_anchors.items():

        p_lat = vp_description["latitude"]
        p_lon = vp_description["longitude"]

        t_lat = target_description["latitude"]
        t_lon = target_description["longitude"]

        d = distance(p_lat, t_lat, p_lon, t_lon)
        distance_probes_target.append((vp_ip, d))

    distance_probes_target = sorted(distance_probes_target, key=lambda d: d[1])
    closest_vp = [vp_ip for vp_ip, _ in distance_probes_target[:nb_vp]]

    # filter out the target ip
    for vp_ip in closest_vp:
        if target_ip in closest_vp:
            raise RuntimeError(
                "the two datasets should not have IP addresses in common")

    return closest_vp

In [12]:
measurements = {}
for target_ip, target_description in target_anchors.items():
    # retrieve the ten closest vp for each targets
    measurements[target_ip] = get_closest_vp(target_ip, target_description, vp_anchors, NB_VP)

    print(f"traget ip: {target_ip}: {target_description}")
    for vp_ip in measurements[target_ip]:
        print(f"probe ip {vp_ip}: {vp_anchors[vp_ip]}")
    print()

traget ip: 207.246.88.250: {'id': 6436, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.5605, 'longitude': -74.4815}
probe ip 23.148.232.6: {'id': 7193, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.5585, 'longitude': -74.4785}
probe ip 144.172.124.251: {'id': 7095, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.6095, 'longitude': -74.1685}
probe ip 92.38.169.178: {'id': 6644, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.7795, 'longitude': -74.0805}
probe ip 170.39.226.151: {'id': 6927, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.7905, 'longitude': -74.0815}
probe ip 162.255.145.7: {'id': 6574, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.7795, 'longitude': -74.0695}
probe ip 104.131.160.184: {'id': 6972, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.7085, 'longitude': -74.0095}
probe ip 152.195.68.35: {'id': 6549, 'is_anchor': True, 'country_code': 'US', 'latitude': 40.7115, 'longitude': -74.0105}
probe ip 217.243.

## start measurement

In [13]:
pinger = PING(RIPE_CREDENTIALS)
dry_run = True

measurement_ids = []
for target_ip in measurements:
    print(f"starting measurement for target {target_ip}")
    print(f"with {len(measurements[target_ip])} vp: {[vp_ip for vp_ip in measurements[target_ip]]}")

    # get anchor ids
    vp_ids = [
        vp_anchors[vp_ip]['id']
        for vp_ip in measurements[target_ip]
    ]
    # run measurement
    measurement_id = pinger.ping_by_target(target_ip, vp_ids,"test", NB_PACKETS, dry_run)
    
    if measurement_id:
        measurement_ids.append(measurement_id)

        print(f"measurement done, measurement id: {measurement_id}")
        print()
    else:
        print("two many concurrent measurements")
        time.sleep(4)

starting measurement for target 207.246.88.250
with 10 vp: ['23.148.232.6', '144.172.124.251', '92.38.169.178', '170.39.226.151', '162.255.145.7', '104.131.160.184', '152.195.68.35', '217.243.179.165', '109.230.208.226', '23.157.160.140']


IndexError: list index out of range